[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/Random.ipynb)

# ランダム推薦
0.5から5の一様乱数を予測評価値とする

In [1]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

# データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

In [1]:
# Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [3]:
# ユーザーIDとアイテムIDに対して、０始まりのインデックスを割り振る
unique_user_ids = sorted(movielens_train.user_id.unique())
unique_movie_ids = sorted(movielens_train.movie_id.unique())
user_id2index = dict(zip(unique_user_ids, range(len(unique_user_ids))))
movie_id2index = dict(zip(unique_movie_ids, range(len(unique_movie_ids))))


In [5]:
import numpy as np
# ユーザー×アイテムの行列で、各セルの予測評価値は0.5〜5.0の一様乱数とする
pred_matrix = np.random.uniform(0.5, 5.0, (len(unique_user_ids), len(unique_movie_ids)))


In [6]:
# rmse評価用に、テストデータに出てくるユーザーとアイテムの予測評価値を格納する
movie_rating_predict = movielens_test.copy()
pred_results = []
for i, row in movielens_test.iterrows():
    user_id = row["user_id"]
    # テストデータのアイテムIDが学習用に登場していない場合も乱数を格納する
    if row["movie_id"] not in movie_id2index:
        pred_results.append(np.random.uniform(0.5, 5.0))
        continue
    # テストデータに現れるユーザーIDとアイテムIDのインデックスを取得し、評価値行列の値を取得する
    user_index = user_id2index[row["user_id"]]
    movie_index = movie_id2index[row["movie_id"]]
    pred_score = pred_matrix[user_index, movie_index]
    pred_results.append(pred_score)
movie_rating_predict["rating_pred"] = pred_results

In [8]:
from collections import defaultdict

# ランキング評価用のデータ作成
# 各ユーザに対するおすすめ映画は、そのユーザがまだ評価していない映画の中からランダムに10作品とする
# キーはユーザーIDで、バリューはおすすめのアイテムIDのリスト
pred_user2items = defaultdict(list)
# ユーザーがすでに評価した映画を取得する
user_evaluated_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()
for user_id in unique_user_ids:
    user_index = user_id2index[user_id]
    movie_indexes = np.argsort(-pred_matrix[user_index, :])
    for movie_index in movie_indexes:
        movie_id = unique_movie_ids[movie_index]
        if movie_id not in user_evaluated_movies[user_id]:
            pred_user2items[user_id].append(movie_id)
        if len(pred_user2items[user_id]) == 10:
            break
pred_user2items

defaultdict(list,
            {1: [38992, 32213, 847, 3932, 6217, 1782, 3592, 2131, 4758, 206],
             2: [5656, 714, 4560, 40946, 2729, 7842, 5569, 4551, 570, 2922],
             3: [32584, 5791, 7027, 7156, 6316, 32770, 7369, 4104, 1407, 8584],
             4: [1244,
              1513,
              2674,
              47778,
              42728,
              6653,
              8659,
              1917,
              1306,
              40946],
             5: [3071, 628, 5322, 2197, 5323, 5802, 6510, 53000, 1377, 3107],
             6: [4310, 589, 26085, 6126, 1880, 6653, 1103, 3401, 29, 6973],
             7: [2018, 990, 1534, 44828, 2949, 2833, 4351, 31685, 1841, 5345],
             8: [959, 49274, 40629, 1980, 3754, 3075, 62113, 1979, 1121, 6196],
             9: [2976, 5936, 51931, 2758, 6554, 309, 8977, 355, 839, 6155],
             10: [7828, 2010, 2983, 1552, 1884, 3634, 5098, 4701, 4970, 3849],
             11: [7307, 7893, 6322, 3083, 2214, 32302, 2488, 2463, 5796,

In [9]:
# user_id=2のユーザーが学習データで評価を付けた映画一覧
movielens_train[movielens_train.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
6150,2,648,2.0,868244699,Mission: Impossible (1996),"[Action, Adventure, Mystery, Thriller]","[confusing, confusing plot, memorable sequence...",12.0
6531,2,733,3.0,868244562,"Rock, The (1996)","[Action, Adventure, Thriller]","[gfei own it, alcatraz, nicolas cage, sean con...",18.0
6813,2,736,3.0,868244698,Twister (1996),"[Action, Adventure, Romance, Thriller]","[disaster, disaster, storm, bill paxton, helen...",13.0
7113,2,780,3.0,868244698,Independence Day (a.k.a. ID4) (1996),"[Action, Adventure, Sci-Fi, War]","[action, alien invasion, aliens, will smith, a...",14.0
7506,2,786,3.0,868244562,Eraser (1996),"[Action, Drama, Thriller]","[arnold schwarzenegger, action, arnold, arnold...",19.0
7661,2,802,2.0,868244603,Phenomenon (1996),"[Drama, Romance]","[interesting concept, own, john travolta, john...",15.0
7779,2,858,2.0,868245645,"Godfather, The (1972)","[Crime, Drama]","[oscar (best picture), marlon brando, classic,...",9.0


In [10]:
# user_id=2に対するおすすめ(5656, 714, 4560)
movies[movies.movie_id.isin([5656, 714, 4560])]

,movie_id,title,genre,tag
702,714,Dead Man (1995),"[Drama, Mystery, Western]","[jim jarmusch, beautiful, iggy pop!!!, johnny ..."
4467,4560,Watchers (1988),"[Horror, Sci-Fi]","[based on a book, dean koontz]"
5558,5656,Festival in Cannes (2001),[Drama],NaN
